
> **ISO2024 INTRODUCTORY SPATIAL 'OMICS ANALYSIS**
>
>
>- HYBRID : TORONTO & ZOOM
>- 10TH JULY 2024 <br>

>**Module 6 : Drawing the boundaries ** <BR>
>
>**Instructor : Shamini Ayyadhury**
>
---

> TOPICS COVERED

* A. Classical segmentation
* B. Segmentation-free

***

```
B. SEGMENTATION-FREE

1. NOW WE WILL PERFORM BAYSOR NON-IMAGE BASED SEGMENTATION
2. BAYSOR IS VERY MEMORY-INTENSIVE - SO WE WILL TEST A TOY DATASET HERE - YES A MUCH SMALLER SAMPLE SIZE THAN THE ALREADY DOWNSIZED TRANSCRIPT SAMPLE.
3. AFTER WHICH, YOU WILL USE THE BAYSOR OUTPUT THAT WE HAVE ALREADY PROCESSED FOR YOU, FOR ALL DOWNSTREAM ANALYSIS

In [ ]:
import os
import subprocess
import scanpy as sc
import pandas as pd
import sys
import seaborn as sns

sys.path.append('/home/shamini/data/projects/spatial_workshop/')
import pre_processing_fnc as ppf

In [ ]:
### directory & filepaths
data_dir = '/home/shamini/data1/data_orig/data/spatial/xenium/10xGenomics/cell_seg_brain_cancer/'
out = '/home/shamini/data/projects/spatial_workshop/out/module4/'


>>> DEMO - STEP 

This step has already been done for you. <br>
See supplementary script 03


In [ ]:
###  BAYSOR SEGMENATION STEP

# get filtered transcript files
### Read in the transcripts_df.csv file from module 2
#os.makedirs(out+'baysor_test', exist_ok=True)
#os.makedirs(out+'baysor_test/segmentation', exist_ok=True)

#command = [
#        '/home/shamini/baysor/bin/baysor/bin/baysor', 'run',
#        '-c', out+'baysor/parameters_baysor.toml',
#        '-s', '9',
#        '--save-polygons', 'geojson',
#        '-o', os.path.join(out, 'baysor_test/segmentation/'),
#        os.path.join(out,'transcripts_subset_all_genes_downsampled.csv')
#        ]
#subprocess.run(command, check=True)

#----------------------------------------------
#ppf.get_memory_usage() ### monitor memory usage

This step has already been done for you. <br>
See supplementary script 03

In [ ]:
### NCV STEP

#os.makedirs(out+'baysor_test/ncv', exist_ok=True)
#command = [
#        '/home/shamini/baysor/bin/baysor/bin/baysor', 'segfree',
#        '-c', out+'baysor/parameters_baysor.toml',
#        '-o', os.path.join(out, 'baysor_test/ncv'),
#        os.path.join(data_dir,'module4/transcripts_subset_all_genes_downsampled.csv')
#        ]
#subprocess.run(command, check=True)

#----------------------------------------------
#ppf.get_memory_usage() ### monitor memory usage

<<< TOY RUN END

>>> Now we will read in the previously processed baysor segmentation files, which is matched with the image we analyzed in our previous script.

In [ ]:
#### load baysor output
baysor_seg = pd.read_csv(out+'baysor/segmentation/scale9/segmentation.csv')


#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

>>> DISCUSSION : THE OUTPUT TABLE BELOW SHOWS SOME NEW ADDITIONS AS WELL AS SOME FAMILAR COLUMNS FROM THE ORIGINAL TRANSCRIPT FILE . 

In [ ]:
baysor_seg

In [ ]:
### TRANSCRIPT ASSIGNMENT 

print('Baysor transcript assignment:\n ', baysor_seg['is_noise'].value_counts(normalize=True)), 
print('\n Xenium transcript assignment:\n', baysor_seg['binary'].value_counts(normalize=True))


WE WILL NOW CLEAN THE BAYSOR SEGMENTATION OUTPUT TO REMOVE UNASSINGED TRANSCRIPTS AND IMPORT THE SEGMENTATION POLYGON JSON FILE FROM THE BAYSOR RUN

In [ ]:

### drop the noise cells
baysor_seg_clean = baysor_seg[baysor_seg['is_noise'] == False]

### get rid of cells with low assignment confidence
baysor_seg_clean = baysor_seg_clean[baysor_seg_clean['assignment_confidence'] > 0.7]

### old and new segmentation files

import json

with open(out+'baysor/segmentation/scale9/segmentation_polygons.json') as f:
    baysor_boundaries = json.load(f)

orig_boundaries = pd.read_csv(out+'cell_boundaries_subset.csv')



UNLIKE THE ORIGINAL XENIUM SEGMENTATION POLYGON FILE, WHICH WAS A DATAFRAME, THE BAYSOR SEGMENTATION POLYGONS ARE STORES AS A JSON FILE. <BR>
SINCE WE REMOVED TRANSCIPTS THAT WERE UNASSIGNED, WE NEED TO ENSURE WE REMOVE ANY EMPTY CELLS AS A RESULT.

In [ ]:
import json

# Extract cell IDs from the DataFrame after converting them to the correct format
matching_cell_ids = baysor_seg_clean['cell'].str.split('-').str[1].astype(int).unique()

# Filter the JSON data to include only the relevant cell IDs
baysor_boundaries_filtered = [geom for geom in baysor_boundaries['geometries'] if geom['cell'] in matching_cell_ids]

# Verify the number of filtered geometries
print("Number of filtered geometries:", len(baysor_boundaries_filtered))

# Create the new JSON structure
filtered_json_data = {'geometries': baysor_boundaries_filtered}

# Convert the filtered data to a JSON string for easier inspection
filtered_json_str = json.dumps(filtered_json_data, indent=4)

# Print the filtered JSON string
print(filtered_json_str)


NOW, LET'S PLOT AND COMPARE THE IMAGE-BASED AND NON-IMAGE-BASED METHODS

In [ ]:
### plotting the segmentation results
### Note that you have been given baysor segmentation carried out on 3 different scales. Plot each and 

import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import tifffile as tiff


iF_crop = tiff.imread(data_dir+'module4/cropped_image_fluo.tif')
composite_image = ppf.plot_composite_image(iF_crop)


fig, ax = plt.subplots(1, 2, figsize=(10, 10))

xlower = 0
ylower = 2000
xlim = [xlower, xlower+1200]
ylim = [ylower, ylower+1200]

### plot the original segmentation

ax[0].imshow(composite_image)

grouped = orig_boundaries.groupby('cell_id')

for cell_id, group in grouped:
    group = pd.concat([group, group.iloc[:1]])
    plg = Polygon(group[['vertex_x', 'vertex_y']].values, edgecolor='red', facecolor='none')
    ax[0].add_patch(plg)

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
ax[0].set_title('Original segmentation')


### now plot the baysor segmentation which is a json file
ax[1].imshow(composite_image)


for geom in filtered_json_data['geometries']:
    plg = geom['coordinates'][0]
    plg.append(plg[0])
    polygon = plt.Polygon(plg, edgecolor='red', facecolor='none', closed=True)
    ax[1].add_patch(polygon)
    ax[1].set_xlim(xlim)
    ax[1].set_ylim(ylim)
ax[1].set_title('Baysor segmentation')
   
ax[1].set_xlim(xlim)
ax[1].set_ylim(ylim)
ax[1].set_title('Baysor segmentation')

     

#----------------------------------------------
ppf.get_memory_usage() ### monitor memory usage

>>> Note that the above baysor segmentation is oversegmented. <br>
>>> If you plot the next scale done, scale=45, it will be under-segmented.<br>

>>> Try repeating the baysor algorithm on your own, using various scaled between 9 and 45.

NOW LET'S EVALUATE THE TRANSCRIPT COMPOSITION AND OVERLAP OF THE FOLLOWING GENES : ANXA1, PTPRC AND STMN1

In [ ]:
import pandas as pd

# List of genes of interest
genes = ['PTPRC', 'ANXA1', 'STMN1']

# Subset the data
baysor_seg_genes = baysor_seg_clean[baysor_seg_clean['gene'].isin(genes)]

# Function to get unique cells expressing specific genes
def get_unique_cells(data, gene, cell_col='cell_id'):
    return data[cell_col][data['gene'] == gene].unique()

# Original/Xenium data
cells_stmn1 = get_unique_cells(baysor_seg_genes, 'STMN1')
cells_anxa1 = get_unique_cells(baysor_seg_genes, 'ANXA1')
cells_ptprc = get_unique_cells(baysor_seg_genes, 'PTPRC')

# Total cells in original/Xenium
total_cells_orig = len(set(cells_stmn1) | set(cells_anxa1) | set(cells_ptprc))

# Cells expressing combinations of genes (original/Xenium)
cells_all = set(cells_stmn1) & set(cells_anxa1) & set(cells_ptprc)
cells_stmn1_anxa1 = set(cells_stmn1) & set(cells_anxa1)
cells_stmn1_ptprc = set(cells_stmn1) & set(cells_ptprc)
cells_anxa1_ptprc = set(cells_anxa1) & set(cells_ptprc)

# Baysor data
cells_stmn1_b = get_unique_cells(baysor_seg_genes, 'STMN1', cell_col='cell')
cells_anxa1_b = get_unique_cells(baysor_seg_genes, 'ANXA1', cell_col='cell')
cells_ptprc_b = get_unique_cells(baysor_seg_genes, 'PTPRC', cell_col='cell')

# Total cells in Baysor
total_cells_baysor = len(set(cells_stmn1_b) | set(cells_anxa1_b) | set(cells_ptprc_b))

# Cells expressing combinations of genes (Baysor)
cells_all_b = set(cells_stmn1_b) & set(cells_anxa1_b) & set(cells_ptprc_b)
cells_stmn1_anxa1_b = set(cells_stmn1_b) & set(cells_anxa1_b)
cells_stmn1_ptprc_b = set(cells_stmn1_b) & set(cells_ptprc_b)
cells_anxa1_ptprc_b = set(cells_anxa1_b) & set(cells_ptprc_b)

# Create a DataFrame to store the normalized counts
comparison_table = pd.DataFrame({
    'Condition': [
        'STMN1', 'ANXA1', 'PTPRC',
        'STMN1 & ANXA1', 'STMN1 & PTPRC', 'ANXA1 & PTPRC',
        'All 3 Genes'
    ],
    'Original/Xenium': [
        len(cells_stmn1) / total_cells_orig, len(cells_anxa1) / total_cells_orig, len(cells_ptprc) / total_cells_orig,
        len(cells_stmn1_anxa1) / total_cells_orig, len(cells_stmn1_ptprc) / total_cells_orig, len(cells_anxa1_ptprc) / total_cells_orig,
        len(cells_all) / total_cells_orig
    ],
    'Baysor': [
        len(cells_stmn1_b) / total_cells_baysor, len(cells_anxa1_b) / total_cells_baysor, len(cells_ptprc_b) / total_cells_baysor,
        len(cells_stmn1_anxa1_b) / total_cells_baysor, len(cells_stmn1_ptprc_b) / total_cells_baysor, len(cells_anxa1_ptprc_b) / total_cells_baysor,
        len(cells_all_b) / total_cells_baysor
    ]
})

# Display the comparison table
print(comparison_table)


```
Always remember that is important to evaluate both the images and the above proportions when evaluating segmentation algorithms
```

>>> DISCUSSION
1. WHAT HAVE WE LEARNT FROM THIS TUTORIAL? 
2. WHICH SEGMENTATION METHOD DO YOU PREFER?
3. DOES THIS MEAN MACHINE LEARNING IMAGE-BASED MODELS ARE BETTER AT SEGMENTATION?
4. WHAT USE DO WE HAVE THEN FOR STATISTICALLY GUIDED TOOLS SUCH AS BAYSOR?

>>> SEG-FREE ANALYSIS

>>> STOP TO EXPLAIN CAREFULLY



1. Baysor also provides a ncv output file.
2. This is the 'neighborhood compositional vector' based on single molecule clustering (transcript clustering and the gene compositions of local clusters)

```
IMPORTANT
```
NCV compute time is long.
The loompy package is not installed.
1. We have extracted the relevant NCV values and saved them.
2. You will be using these pre-computed values and plotting these

The following code has already been run for you. 

---
import loompy <br>

### read loom file 
lm = loompy.connect(out+'baysor/ncv/ncvs.loom') <br>

lm_ptprc = lm[lm.ra['Name']=='PTPRC',:].T <br>
lm_stmn1 = lm[lm.ra['Name']=='STMN1',:].T <br>
lm_anxa1 = lm[lm.ra['Name']=='ANXA1',:].T <br>

baysor_seg['ncv_ptprc'] = lm_ptprc <br>
baysor_seg['ncv_stmn1'] = lm_stmn1 <br>
baysor_seg['ncv_anxa1'] = lm_anxa1 <br>

lm.close() <br>

baysor_seg.to_csv(out+'baysor/ncv/baysor_seg_ncv.csv', index=False) <br>

---

* Read the pre-computed baysor segmentation file that aready has the ncv vectors for PTPRC, ANXA1 and STMN1 appended
* We will drop the low confidence cells

In [ ]:
### Look at the baysor_seg file that has the ncv details for PTPC, ANXA1 qnd STMN1 appended

baysor_seg = pd.read_csv(out+'baysor/ncv/baysor_seg_ncv.csv')
baysor_seg



In [ ]:

### drop the noise cells
baysor_seg_clean = baysor_seg[baysor_seg['is_noise'] == False]

### get rid of cells with low assignment confidence
baysor_seg_clean = baysor_seg_clean[baysor_seg_clean['assignment_confidence'] > 0.7]



In [ ]:
baysor_seg_downsampled = baysor_seg_clean[(baysor_seg_clean['x'] < 600) & (baysor_seg_clean['y'] < 600)].copy()
baysor_seg_downsampled

In [ ]:

from matplotlib import patches as mpatches

### to save on compute time, we will plot only a fraction of the transcripts 
frac = 0.25

### subset only ncv values for the genes of interest, greater than zero
baysor_seg_downsampled_ptprc = baysor_seg_downsampled[baysor_seg_downsampled['ncv_ptprc'] > 0]
baysor_seg_downsampled_stmn1 = baysor_seg_downsampled[baysor_seg_downsampled['ncv_stmn1'] > 0]
baysor_seg_downsampled_anxa1 = baysor_seg_downsampled[baysor_seg_downsampled['ncv_anxa1'] > 0]

### define the subplots 
fig, axes = plt.subplots(2, 2, figsize=(9, 7))
axes = axes.flatten()

### first we group the values by ncv color - recall lecture
grouped = baysor_seg_downsampled.groupby('ncv_color')

### PLOT 1 - here will plot all transcripts by their pre-calculated NCV color values
for name, group in grouped:
    group = group.sample(frac=frac)
    sns.scatterplot(group, x='x', y='y', color= group['ncv_color'].unique(), ax=axes[0], s=1, rasterized=True)
sns.despine()
axes[0].set_title('All transcripts by NCV color')

### PLOT 2 - on the adjacent plot, we will the plot the gene expression distribution for the three genes of interest
ax = sns.scatterplot(data=baysor_seg_downsampled_ptprc, x='x', y='y', hue='ncv_ptprc', s=1, ax=axes[1], palette='Greens')
ax.legend().remove()
ax = sns.scatterplot(data=baysor_seg_downsampled_stmn1, x='x', y='y', hue='ncv_stmn1', s=1, ax=axes[1], palette='Reds')
ax.legend().remove()
ax = sns.scatterplot(data=baysor_seg_downsampled_anxa1, x='x', y='y', hue='ncv_anxa1', s=1, ax=axes[1], palette='Blues')
ax.legend().remove()
ax.set_title('Gene expression distribution')

# PLOT 3 - Same as plot 1 but a zoomed-in section
for name, group in grouped:
    group = group.sample(frac=frac)
    sns.scatterplot(group, x='x', y='y', color= group['ncv_color'].unique(), ax=axes[2], s=9, rasterized=True)
sns.despine()
axes[2].set_xlim(200, 500)
axes[2].set_ylim(300, 600)  
axes[2].set_title('All transcripts by NCV color (zoomed-in)')

# PLOT 4 - Same as plot 2 but a zoomed-in section
ax = sns.scatterplot(data=baysor_seg_downsampled_ptprc, x='x', y='y', hue='ncv_ptprc', s=9, ax=axes[3], palette='Greens')
ax.legend().remove()
ax = sns.scatterplot(data=baysor_seg_downsampled_stmn1, x='x', y='y', hue='ncv_stmn1', s=9, ax=axes[3], palette='Reds')
ax.legend().remove()
ax = sns.scatterplot(data=baysor_seg_downsampled_anxa1, x='x', y='y', hue='ncv_anxa1', s=9, ax=axes[3], palette='Blues')
ax.legend().remove()
axes[3].set_xlim(200, 500)
axes[3].set_ylim(300, 600)  # Invert y-axis for the scatter plot
axes[3].set_title('Gene expression distribution (zoomed-in)')

# Create custom legend
green_patch = mpatches.Patch(color='green', label='PTPRC')
red_patch = mpatches.Patch(color='red', label='STMN1')
blue_patch = mpatches.Patch(color='blue', label='ANXA1')

plt.legend(handles=[green_patch, red_patch, blue_patch], title='Gene expression', loc='upper right', bbox_to_anchor=(1.5, 1))

sns.despine()
plt.tight_layout()
plt.show()

CONCLUDING DISCUSSION
1. What are the benefits of segmentation-free analysis?
2. Can we replace segmentation with these?
3. What are some of the real world use case?

>>> END OF MODULE 4
>>> SEE YOU NEXT for MODULE 5 & 6